In [2]:
!pip install -q transformers einops
!pip install transformers[torch]
!pip install git+https://github.com/huggingface/peft.git
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 10.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-38qpp4gy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-38qpp4gy
  Resolved https://github.com/huggingface/peft.git to commit d17266d59923948021dacc85bd08489845c4389d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.6.0.dev0-py3-none-any.whl size=122165 

In [3]:
!pip install --upgrade bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 18.4 MB/s eta 0:00:00


Load model and necessary configurations

In [4]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = "vilsonrodrigues/falcon-7b-sharded"

#Quantizer initialize
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

#Get model from HuggingFace's transformers library
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Load and preprocess dataset

In [40]:
from datasets import load_dataset

dataset_name1 = "pbaoo2705/processed_dataset_v2"
dataset_name2 = "pubmed_qa"
#dataset = load_dataset(dataset_name2, 'pqa_artificial')

'''
formatted_dataset = dataset["train"].train_test_split(test_size=1000, train_size=4000)
train_dataset = formatted_dataset["train"]
eval_dataset = formatted_dataset["test"]
'''

train_dataset = load_dataset(dataset_name1, split='train')
eval_dataset = load_dataset(dataset_name1, split='test')

In [6]:
from transformers import AutoTokenizer, AutoModelWithLMHead, GPT2Tokenizer

#Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

#tokenizer_sum = AutoTokenizer.from_pretrained('T5-small')
#model_sum = AutoModelWithLMHead.from_pretrained('T5-small', return_dict=True)

In [42]:
train_dataset = train_dataset.map(lambda row: {"input_ids": tokenizer(row["text"], padding="max_length", truncation=True, max_length=512).input_ids}, batched=True)
train_dataset = train_dataset.remove_columns(["pubid"])
eval_dataset = eval_dataset.map(lambda row: {"input_ids": tokenizer(row["text"], padding="max_length", truncation=True, max_length=512).input_ids}, batched=True)
eval_dataset = eval_dataset.remove_columns(["pubid"])



Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [43]:
train_dataset

Dataset({
    features: ['question', 'context', 'long_answer', 'final_decision', 'text', 'input_ids'],
    num_rows: 5000
})

Prefix tuning configuration

In [9]:
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType

peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
model = get_peft_model(model, peft_config)

In [10]:
model.print_trainable_parameters()

trainable params: 8,724,480 || all params: 6,930,445,184 || trainable%: 0.12588628534486943


In [46]:
from transformers import default_data_collator, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=8, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=8, pin_memory=True)

In [12]:
len(train_dataloader)

625

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * 5),
)

Set up training

In [14]:
#!pip install wandb

In [15]:
#!wandb login

In [16]:
#import wandb

#wandb.init(project="prefix-tuning-evaluation")

In [17]:
from transformers import TrainingArguments

#Arguments needed for training process
output_dir = "falcon-7b-sharded"
#arguments set up on Huggingface
per_device_train_batch_size = 8
gradient_accumulation_steps = 4
device = "cuda"
num_epochs = 5
#Decoupled weight regularisation optimiser recommended by paper
optim = "adamw_torch"
save_steps = 10
logging_steps = 10
learning_rate = 1e-2
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0
lr_scheduler_type = "linear"


training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_epochs,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    report_to=None,
    lr_scheduler_type=lr_scheduler_type

)


In [18]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    dataset_text_field="text",
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:166: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

In [47]:
from tqdm import tqdm

model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")


  0%|          | 0/625 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
trainer.train()